In [1]:
import numpy as np
from pathlib import Path

## Isolation des mots

En tout premier lieu, on veut isoler les mots les uns des autres pour pouvoir les manipuler. Certains caractères de ponctuation, comme les deux points ou la virgule, seront volontairement traités comme des mots pour fluidifier les phrases et simplifier les programmes.

In [2]:
cwd = Path.cwd().as_posix() + '/textsFolder/'

def read_text(fileName):
    """Fonction pour lire un document .txt"""
    with open(cwd + fileName, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

In [3]:
def text_to_list(fileName):
    """Converti un .txt en une liste de mots et de ponctuation, en gardant le même ordre"""
    text = read_text(fileName)
    word = str()
    end_word = [' ', '\n', '.', ',', '!', ':', '?', '(', ')', '/', '*', "'", '-'] #n'importe quel caractère qui assure qu'on est à la fin d'un mot
    list_of_words = list()
    
    for i in range(len(text)):
        if text[i] in end_word:
            if word != str():
                list_of_words.append(word)
                word = str()
            if text[i] in ['.', '?', '!', ',', ':', ';']: #si on touche à cette liste, on doit changer de matrices
                list_of_words.append(text[i])
        else:
            word += text[i].lower() # convertit tous les caractères en minuscule
    return list_of_words

## Construction de la matrice de probabilité

L'idée ici est de créer une liste de tous les mots que nous pouvons rencontrer, avec une seule fois chaque mot.
On peut ensuite créer une matrice de probabilités du passage d'un mot au suivant. On étendra ensuite le procédé en tenant compte des n mots précédents.

In [4]:
def sort_list(text):
    """Trie une list par ordre alphabétique et supprime les doublons"""
    return sorted(list(set(text_to_list(text))))

#On vire les éléments redondants en passant par un Set, et on trie par ordre alphabétique avec la fonction sorted()


### Matrice des n-1
Dans un premier temps, on ne regarde que la probabilité de passage d'un mot au suivant

Création de la matrice:

In [5]:
def give_probability_matrix_1(text = 'test4.txt'):
    """Renvoie la matrice de probabilité en tenant compte de n-1"""
    list_of_words = sort_list(text)
    n = len(list_of_words)
    matrix = np.zeros((n,n))
    for i in range(len(text_to_list(text)) - 1):
        idx_word = list_of_words.index(text_to_list(text)[i]) # on cherche l'emplacement du mot dans la liste de mots bien rangée
        idx_next_word = list_of_words.index(text_to_list(text)[i+1])
        matrix[idx_word, idx_next_word] += 1
    return matrix
    
    

In [6]:
def saveMatrix(matrix, fileName): # on enregistre la matrice pour pouvoir la réutiliser sans avoir à la recalculer
    np.save(fileName, matrix)

In [7]:
#matrix1 = give_probability_matrix_1('test4.txt')
#saveMatrix(matrix1, 'matrix1.npy')
# une fois la matrice générée, on ne veut plus toucher à ces fonctions

### Matrice des n-2
On modifie les programmes précédents de manière à prendre en compte les 2 mots précédents pour le calcul des probabilités du mot suivant

In [8]:
def assort_consecutive_words(text):
    """Convertit un text en tuples de mots consécutifs (chaque mot est dans un tuple avec le précédent et le suivant)"""
    list_of_words = text_to_list(text)
    return [(list_of_words[i], list_of_words[i+1]) for i in range(len(list_of_words)-1)]

In [9]:
def give_probability_matrix_2(text = 'test4.txt'):
    """Renvoie la matrice de probabilité en tenant compte de n-1 et n-2"""
    list_of_two_words = assort_consecutive_words(text)
    sorted_list_of_two_words = sorted(list(set(list_of_two_words)))
    n = len(sorted_list_of_two_words)
    matrix = np.zeros((n,n))
    for i in range(n - 1):
        idx_word = sorted_list_of_two_words.index(list_of_two_words[i]) # on cherche l'emplacement du mot dans la liste de mots bien rangée
        idx_next_word = sorted_list_of_two_words.index(list_of_two_words[i+1])
        matrix[idx_word, idx_next_word] += 1
    return matrix

A présent, une fois les matrices calculées, on n'aura qu'à exécuter les programmes en dessous

## Utilisation de la matrice de probabilité

In [10]:
def load_matrix(fileName):
    return np.load(fileName)

In [11]:
matrix = load_matrix('matrix1.npy')

In [12]:

def get_all_first_words(text = 'test4.txt'):
    """Renvoie la liste de tous les mots qui débutent une phrase"""
    list_of_words = text_to_list(text)
    l = list()
    l.append(list_of_words[0])
    for i in range(1, len(list_of_words)):
        if list_of_words[i-1] in ['.', '?', '!']:
            l.append(list_of_words[i])
    return l

In [13]:
def sentence_generator1(matrix, text = 'test4.txt'):
    """Renvoie une phrase générée aléatoirement à partir d'une matrice de probabilités
    La probabilité de choix d'un mot ne dépend que du mot précédent"""
    sentence = list()
    word = np.random.choice(get_all_first_words(text))
    sentence.append(word)
    while word not in ['.', '?', '!']:
        idx = sort_list(text).index(word)
        word = np.random.choice(sort_list(text),1, p=matrix[idx]/np.sum(matrix[idx]))
        sentence.append(word[0])
    sentence_in_str = str()
    for word in sentence:
        sentence_in_str += word + ' '
    return sentence_in_str

#### Programme réajusté pour n = 2
On réajuste le programme pour que la proba d'un mot prenne en compte les deux mots précédents

In [14]:
def sort_list_of_set(text = 'test4.txt'):
    """renvoie une liste ordonée sans doublons de tuples de mots consécutifs issus du texte"""
    list_of_one_word = text_to_list(text)
    return sorted(list(set([(list_of_one_word[i], list_of_one_word[i+1]) for i in range(len(list_of_one_word)-1)])))

In [15]:
def get_all_first_two_words(text = 'test4.txt'):
    """On se servira de cette fonction pour commencer nos phrase avec un premier mot crédible. Cette fonction renvoie donc la liste de tous les mots qui débutent une phrase"""
    list_of_simple_words = text_to_list(text)
    list_of_two_words = [(list_of_simple_words[i], list_of_simple_words[i+1]) for i in range(len(list_of_simple_words)-1)]
    l = list()
    l.append(list_of_two_words[0])
    for i in range(1, len(list_of_two_words)):
        if list_of_two_words[i-1][0] in ['.', '?', '!']:
            l.append(list_of_two_words[i])
    return l

In [16]:
def sentenceGenerator2(matrix, text = 'test4.txt'):
    """Génère aléatoirement des phrases d'après une matrice de probabilité qui tient compte des deux précédents mots"""
    sentence = list()
    i = np.random.randint(0,len(get_all_first_two_words(text)),1)[0]
    words = get_all_first_two_words(text)[i]
    sentence.append(words[0])
    words_set = sort_list_of_set(text)
    while words[-1] not in ['.', '?', '!']:
        idx = words_set.index(words)
        i = np.random.choice(len(words_set),1, p=matrix[idx]/np.sum(matrix[idx]))[0]
        words = words_set[i]
        sentence.append(words[0])
    sentence_as_str = str()
    for word in sentence:
        sentence_as_str += str(word) + ' '
    return sentence_as_str

In [17]:
matrix2 = give_probability_matrix_2('test4.txt')

Et voilà le résultat final:

In [18]:
sentenceGenerator2(matrix2, 'test4.txt')

'ils se précipitaient de leurs maisons pour courir en foule à la porte d une mauvaise hôtellerie , devant laquelle étaient deux chariots couverts '

### Conclusion
On voit que le programme est très limité. La plupart du temps, il ne fait que rabouter deux phrases préexistantes. Cependant, avec une meilleure puissance de calcul, il serait possible de construire de plus grandes matrices de probabilités, tenant compte de plus de mots précédents pour le choix du mot suivant (pour assurer des phrases cohérentes) et d'un plus grand jeu de données (pour des phrases plus détachées du texte de référence).